In [1]:
from datasets import load_dataset
from evaluate import evaluator
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline

2023-11-20 11:59:44.700028: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 11:59:44.718782: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 11:59:44.718798: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 11:59:44.718811: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 11:59:44.722293: I tensorflow/core/platform/cpu_feature_g

In [8]:
imagenet = load_dataset("imagenet-1k", split="validation[:1000]")

In [3]:
task_evaluator = evaluator("image-classification")

In [4]:
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-giant-imagenet1k-1-layer')
model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-giant-imagenet1k-1-layer')
pipe = pipeline(
            task="image-classification",
            model=model,
            image_processor=processor,
            device="cuda:0"
        )

In [5]:
resnet = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50")
label2id = resnet.config.label2id
del resnet

In [9]:
%%time
results = task_evaluator.compute(
                model_or_pipeline=pipe,
                data=imagenet,
                metric="accuracy",
                label_mapping=label2id)

CPU times: user 2min 49s, sys: 222 ms, total: 2min 49s
Wall time: 33.1 s


In [10]:
results

{'accuracy': 0.858,
 'total_time_in_seconds': 32.639526112005115,
 'samples_per_second': 30.637699719304162,
 'latency_in_seconds': 0.032639526112005116}